In [ ]:
import json
import random
import numpy as np

def stable_softmax(x):
    z = x - max(x)
    numerator = np.exp(z)
    denominator = np.sum(numerator)
    softmax = numerator/denominator

    return softmax

In [ ]:
with open('BBGM_League_1_2020_re_sign_players.json','r',encoding='utf-8-sig') as fp:
    txt = fp.read()

In [ ]:
data = json.loads(txt)

In [ ]:
data.keys()

In [ ]:
for p in data['players']:
    p['freeAgentMood']= 30*[0]
for t in data['gameAttributes']:
    if t['key'] == 'salaryCap':
        cap = t['value']
    if t['key'] == 'minContract':
        minS = t['value']
    if t['key'] == 'maxContract':
        maxS = t['value']
numTeams = len(data['teams'])

In [ ]:
RAND_NUM = 128 # power of 2 plz
RAND_NOISE = 1.5
RN = RAND_NUM - 1 # mod 2 into hash
rvec = np.random.randn(RAND_NUM)*RAND_NOISE

TEMP = 0.5
SCALE_UP = 1.1
SCALE_DOWN = 0.9
ROUNDS = 30
SCALE_CAP = 1.0 # scaling
ADD_CAP = maxS*0.30 # exception salary
USE_OWN_INIT = True

In [ ]:
players = {}
for p in data['players']:
    if p['tid'] != -1:
        continue
        
    # i dunno, good enough?
    val = p['ratings'][-1]['ovr']*0.5 + p['ratings'][-1]['pot']*0.5
    
    # basically ws/48 prediction from OVR
    val_or = (1/209.33) * (max(val,29.14) - 29.14) ** 2
    
    apy = p['contract']['amount']
    players[p['pid']] = (val_or,apy)

if USE_OWN_INIT:
    p_keys = list(players.keys())
    p_vals = [players[k][1] for k in p_keys]
    # 0 is highest, etc. 
    order = len(p_vals)-1 - np.argsort(p_vals)

    # set order by ourselves simply based on order and number of teams!
    for k,v,o in zip(p_keys,p_vals,order):
        nv = round(maxS*np.exp(-o/numTeams)+minS)
        players[k] = (players[k][0],nv)

In [ ]:
from collections import defaultdict
teams = defaultdict(list)
for p in data['players']:
    if p['tid'] >= 0:
        teams[p['tid']].append(p['contract']['amount'])


In [ ]:
team_totals = {k:sum(v) for k,v in teams.items()}
for t in data['teams']:
    team_totals[t['tid']] = team_totals.get(t['tid'],0)

In [ ]:
#team_totals = {k:0 for k,v in team_totals.items()}
team_totals

In [ ]:
for i in range(ROUNDS):
    # teams bid on players

    bids = defaultdict(int)
    team_data = list(team_totals.items())
    random.shuffle(team_data)
    for t,c in team_data: # randomize team order
        budget = SCALE_CAP*max(0,cap-c)+ADD_CAP
        selected = set([pid for pid,p in players.items() if bids[pid] > 1]) # skip players who we know get signed at their current price
        #selected = set()
        while budget > 0:
            # get valid players
            valid_p = [(p[0]+rvec[hash((pid,t))&RN],pid) for pid,p in players.items() if p[1]<=budget and pid not in selected]

            if len(valid_p) > 0:
                # sample one
                vpa = np.array(valid_p)
                probs = stable_softmax(vpa[:,0]*TEMP)
                pid = int(np.random.choice(vpa[:,1],1,p=probs))

                # act accordingly
                selected.add(pid)
                bids[pid] = 1 + bids[pid]
                budget -= players[pid][1]
            else:
                break
                
    # players adjust expectations
    for pid,v in players.items():
        if pid in bids and bids[pid] > 1:
            players[pid]  = (v[0],min(maxS,v[1]*SCALE_UP))
        elif bids[pid] == 0:
            players[pid] = (v[0],max(minS,v[1]*SCALE_DOWN))
            #print(players[pid],pid)

In [ ]:
res = []
for p in data['players']:
    if p['tid'] != -1:
        continue
    res.append((p['contract']['amount'],players[p['pid']][1]))

In [ ]:
import matplotlib.pyplot as plt
plt.subplot(1,2,1)
plt.hist(vpa[:,0])
plt.subplot(1,2,2)
plt.hist(probs)

In [ ]:
plt.scatter(np.array(res)[:,0],np.array(res)[:,1])
plt.xlabel('old contract')
plt.ylabel('new contract')
plt.title('changes')
plt.tight_layout()

In [ ]:
res2 = res


In [ ]:
plt.plot(sorted(np.array(list(players.values()))[:,1],reverse=True))
xn = np.linspace(0,1000,1000)
plt.plot(xn,maxS*np.exp(-xn/numTeams)+minS)
plt.xlim(0,100)
plt.ylim(0,45000)

In [ ]:
plt.style.use('fivethirtyeight')
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.scatter(np.array(res)[:,0],np.array(res)[:,1])
plt.xlabel('old contract')
plt.ylabel('new contract')
plt.title('high temp (TEMP = 0.1)')
plt.subplot(1,2,2)

plt.scatter(np.array(res2)[:,0],np.array(res2)[:,1])
plt.xlabel('old contract')
plt.ylabel('new contract')
plt.title('low temp (TEMP = 5)')
plt.tight_layout()

plt.suptitle('"whole league is FA" re-auction',y=1.05,size=25)

In [ ]:
change = np.array(res)[:,0]-np.array(res)[:,1]
for i in np.argsort(change):
    print(change[i], data['players'][i]['lastName'],res[i])

In [ ]:
for i,e in enumerate(data['players']):
    if e['pid'] in players:
        data['players'][i]['contract']['amount'] = int(round(players[e['pid']][1]))

In [ ]:
with open('fa_fixed.json','w',encoding='utf-8-sig') as fp:
    json.dump(data,fp)

In [ ]:
data['players'][0]